<a href="https://colab.research.google.com/github/AlkaidCheng/GSoC2021_QMLHEP/blob/main/examples/QuplePQC_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.4.1
!pip install tensorflow-quantum
!pip install quple

     |████████████████████████████████| 394.3MB 39kB/s 
     |████████████████████████████████| 471kB 41.0MB/s 
     |████████████████████████████████| 3.8MB 25.1MB/s 
     |████████████████████████████████| 2.9MB 30.6MB/s 
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Found existing installation: grpcio 1.34.1
    Uninstalling grpcio-1.34.1:
      Successfully uninstalled grpcio-1.34.1
  Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Found existing installation: tensorflow 2.5.0
    Uninstalling tensorflow-2.5.0:
      Successfully uninstalled tensorflow-2.5.0
     |████████████████████████████████| 7.8MB 6.8MB/s 
     |████████████████████████████████| 5.6MB 34.1MB/s 
     |███████████████████

     |████████████████████████████████| 102kB 5.4MB/s 


In [25]:
import sympy
import quple
import cirq
import tensorflow as tf

In [4]:
# let's make some sympy expressions
feature_dimension = 4
x = sympy.symarray("x", feature_dimension)
formula = sympy.pi*(sympy.cos(x[0])*sympy.sin(x[1]))**2/(x[2]+x[3])
formula

pi*sin(x_1)**2*cos(x_0)**2/(x_2 + x_3)

In [5]:
symbols = [i.name for i in x]
symbols

['x_0', 'x_1', 'x_2', 'x_3']

In [6]:
from quple.interface.tfq.tf_resolvers import resolve_formula
# resolve the formula as tf operations
resolved_formula = resolve_formula(formula, symbols)

In [7]:
# try to get the value of the formula from a tensor variable
x = tf.Variable(tf.random.normal((feature_dimension,)))
x

<tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([-0.77177674, -0.03157092, -0.7115509 ,  0.7681052 ], dtype=float32)>

In [8]:
resolved_formula(x)

<tf.Tensor: shape=(), dtype=float32, numpy=0.028428633>

In [9]:
# you can also use 2d tensor, but it will get resolved like
# sin(x[1])*cos(x[0])**x[2] / (x[3]+x[4])
# so if your tensor shape is (batchsize, features), you will need to transpose it first, then transpose it back

# fortunately, there is another method to achieve that without much trouble:
# this method works on a list of formulas
from quple.interface.tfq.tf_resolvers import resolve_formulas
resolved_formulas = resolve_formulas([formula], symbols)

In [10]:
batchsize = 10
x = tf.Variable(tf.random.normal((batchsize, feature_dimension)))
x

<tf.Variable 'Variable:0' shape=(10, 4) dtype=float32, numpy=
array([[-1.0618973 , -0.15144867, -0.8085656 , -0.6831694 ],
       [-0.60386324,  0.76668376,  1.4801891 , -0.29068053],
       [ 0.13388175, -0.5763278 ,  1.7136308 , -0.61749834],
       [ 0.58188254, -1.9518006 ,  0.33640802, -0.363397  ],
       [ 1.1470926 ,  2.3602638 , -0.4549302 ,  0.77253604],
       [-0.11804762, -0.6053382 , -0.22556916,  0.759861  ],
       [ 0.09652334, -1.0528183 , -1.6297547 , -0.6007788 ],
       [-2.558616  ,  1.1557344 , -0.2685567 ,  0.7381837 ],
       [-0.8558756 ,  0.5515809 ,  0.13191238, -0.36481592],
       [ 0.82903636,  1.0982615 ,  0.5969692 , -0.0608829 ]],
      dtype=float32)>

In [11]:
resolved_formulas(x)

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[-1.13791395e-02],
       [ 8.61281455e-01],
       [ 8.35968852e-01],
       [-7.00086746e+01],
       [ 8.29202831e-01],
       [ 1.87754858e+00],
       [-1.05329573e+00],
       [ 3.90408611e+00],
       [-1.59189677e+00],
       [ 2.12062740e+00]], dtype=float32)>

## Practical Example

In [12]:
n_qubit = feature_dimension = 5

In [13]:
from quple import ParameterisedCircuit
model_circuit = ParameterisedCircuit(n_qubit=n_qubit, copies=2, rotation_blocks=['RZ', 'RY'], entanglement_blocks=['XX'], entangle_strategy="alternate_linear")
model_circuit

(0, 0): ───Rz(θ_0)───Ry(θ_5)───XX──────────Rz(θ_14)────Ry(θ_19)──────────────XX──────────────────────
                               │                                             │
(0, 1): ───Rz(θ_1)───Ry(θ_6)───XX^(θ_10)───XX──────────Rz(θ_15)───Ry(θ_20)───XX^(θ_24)───XX──────────
                                           │                                             │
(0, 2): ───Rz(θ_2)───Ry(θ_7)───XX──────────XX^(θ_12)───Rz(θ_16)───Ry(θ_21)───XX──────────XX^(θ_26)───
                               │                                             │
(0, 3): ───Rz(θ_3)───Ry(θ_8)───XX^(θ_11)───XX──────────Rz(θ_17)───Ry(θ_22)───XX^(θ_25)───XX──────────
                                           │                                             │
(0, 4): ───Rz(θ_4)───Ry(θ_9)───────────────XX^(θ_13)───Rz(θ_18)───Ry(θ_23)───────────────XX^(θ_27)───

In [14]:
from quple.data_encoding import EncodingCircuit
from quple import PauliBlock
data_circuit = EncodingCircuit(feature_dimension=feature_dimension, copies=2, rotation_blocks=['RZ'], entanglement_blocks=[PauliBlock('ZZ')], entangle_strategy="alternate_linear", 
                               encoding_map="cosine_product",
                               parameter_scale=1,      # by default will multiply every parameter by sympy.pi if parameter_scale is not specified
                               flatten_circuit=False)  # flattening the circuit will combine formulas into a single symbol so when you print the circuit it doesn't look quite good
data_circuit

(0, 0): ───Rz(x_0)───@───────────────────────────@───────────────────────────────────────────────────────────────────────────────Rz(x_0)───@─────────────────────────────────@───────────────────────────────────────────────────────────────────────────────────
                     │                           │                                                                                         │                                 │
(0, 1): ───Rz(x_1)───X───Rz(cos(x_0)*cos(x_1))───X───────────────────────────@───────────────────────────@───────────────────────Rz(x_1)───X─────────Rz(cos(x_0)*cos(x_1))───X───────────────────────────@───────────────────────────@───────────────────────────
                                                                             │                           │                                                                                               │                           │
(0, 2): ───Rz(x_2)───@───────────────────────────────────────────────────@───X───Rz(cos(x_1)*cos(x_2))───X───────────────────────Rz(x_2)─────────────@───────────────────────────────────────────────@───X───Rz(cos(x_1)*cos(x_2))───X───────────────────────────
                     │                                                   │                                                                           │                                               │
(0, 3): ───Rz(x_3)───X───────────────────────────Rz(cos(x_2)*cos(x_3))───X───@───────────────────────────────────────────────────@─────────Rz(x_3)───X───────────────────────Rz(cos(x_2)*cos(x_3))───X───@───────────────────────────────────────────────────@───
                                                                             │                                                   │                                                                       │                                                   │
(0, 4): ───Rz(x_4)───────────────────────────────────────────────────────────X───────────────────────────Rz(cos(x_3)*cos(x_4))───X─────────Rz(x_4)───────────────────────────────────────────────────────X───────────────────────────Rz(cos(x_3)*cos(x_4))───X───

In [15]:
symbols = data_circuit.symbols
symbols

['x_0', 'x_1', 'x_2', 'x_3', 'x_4']

In [16]:
data_circuit.flatten()
data_circuit

(0, 0): ───Rz(pi*<x_0/pi>)───@───────────────────────────────────@───────────────────────────────────────────────────────────────────────────────────────────────────────Rz(pi*<x_0/pi>)───@─────────────────────────────────────────────────@───────────────────────────────────────────────────────────────────────────────────────────────────────────
                             │                                   │                                                                                                                         │                                                 │
(0, 1): ───Rz(pi*<x_1/pi>)───X───Rz(pi*<cos(x_0)*cos(x_1)/pi>)───X───────────────────────────────────@───────────────────────────────────@───────────────────────────────Rz(pi*<x_1/pi>)───X─────────────────Rz(pi*<cos(x_0)*cos(x_1)/pi>)───X───────────────────────────────────@───────────────────────────────────@───────────────────────────────────
                                                                                                     │                                   │                                                                                                                                       │                                   │
(0, 2): ───Rz(pi*<x_2/pi>)───@───────────────────────────────────────────────────────────────────@───X───Rz(pi*<cos(x_1)*cos(x_2)/pi>)───X───────────────────────────────Rz(pi*<x_2/pi>)─────────────────────@───────────────────────────────────────────────────────────────@───X───Rz(pi*<cos(x_1)*cos(x_2)/pi>)───X───────────────────────────────────
                             │                                                                   │                                                                                                           │                                                               │
(0, 3): ───Rz(pi*<x_3/pi>)───X───────────────────────────────────Rz(pi*<cos(x_2)*cos(x_3)/pi>)───X───@───────────────────────────────────────────────────────────────────@─────────────────Rz(pi*<x_3/pi>)───X───────────────────────────────Rz(pi*<cos(x_2)*cos(x_3)/pi>)───X───@───────────────────────────────────────────────────────────────────@───
                                                                                                     │                                                                   │                                                                                                       │                                                                   │
(0, 4): ───Rz(pi*<x_4/pi>)───────────────────────────────────────────────────────────────────────────X───────────────────────────────────Rz(pi*<cos(x_3)*cos(x_4)/pi>)───X─────────────────Rz(pi*<x_4/pi>)───────────────────────────────────────────────────────────────────────X───────────────────────────────────Rz(pi*<cos(x_3)*cos(x_4)/pi>)───X───

In [17]:
# you can see that flatten() literally flatten the symbols
data_circuit.symbols

['<cos(x_0)*cos(x_1)/pi>',
 '<cos(x_1)*cos(x_2)/pi>',
 '<cos(x_2)*cos(x_3)/pi>',
 '<cos(x_3)*cos(x_4)/pi>',
 '<x_0/pi>',
 '<x_1/pi>',
 '<x_2/pi>',
 '<x_3/pi>',
 '<x_4/pi>']

In [18]:
# by doing flatten(), an cirq.ExpressionMap is created which gives you a handle of the sympy formulas involved
data_circuit.expr_map

cirq.ExpressionMap({x_0/pi: <x_0/pi>, x_1/pi: <x_1/pi>, x_2/pi: <x_2/pi>, x_3/pi: <x_3/pi>, x_4/pi: <x_4/pi>, cos(x_0)*cos(x_1)/pi: <cos(x_0)*cos(x_1)/pi>, cos(x_2)*cos(x_3)/pi: <cos(x_2)*cos(x_3)/pi>, cos(x_1)*cos(x_2)/pi: <cos(x_1)*cos(x_2)/pi>, cos(x_3)*cos(x_4)/pi: <cos(x_3)*cos(x_4)/pi>})

In [19]:
formulas = list(data_circuit.expr_map)
formulas

[x_0/pi,
 x_1/pi,
 x_2/pi,
 x_3/pi,
 x_4/pi,
 cos(x_0)*cos(x_1)/pi,
 cos(x_2)*cos(x_3)/pi,
 cos(x_1)*cos(x_2)/pi,
 cos(x_3)*cos(x_4)/pi]

In [20]:
# you can then resolve the formulas in terms of tf operations using
resolved_formulas = resolve_formulas(formulas, symbols)

In [21]:
# create your favourite training input
batchsize = 10
x = tf.random.normal((batchsize, feature_dimension))
x

<tf.Tensor: shape=(10, 5), dtype=float32, numpy=
array([[-0.6822332 , -0.33362395,  0.1343539 ,  0.14878808,  1.5183932 ],
       [ 1.8541712 ,  1.3194737 , -0.9088058 ,  0.6791304 ,  0.00924885],
       [-1.1402118 ,  1.5177584 , -0.0904963 ,  1.7034129 , -0.2984075 ],
       [-0.82028824, -0.83841646,  1.4964039 ,  0.15012872, -0.71458745],
       [ 0.7315398 , -0.13146573,  1.0615153 ,  2.713292  ,  0.15342617],
       [ 0.10406739, -0.16797851, -0.38000715,  0.60419124,  1.1796423 ],
       [-1.2041612 ,  1.2046661 ,  0.05035108,  0.7293226 , -0.19597055],
       [ 1.3057795 ,  1.1946394 ,  0.19621813,  0.12937541,  0.06831659],
       [-0.34043592,  1.7567332 ,  0.9004694 , -0.64915144,  0.12494518],
       [-0.58868444, -0.22680373, -0.8886248 ,  1.2729706 ,  0.7304239 ]],
      dtype=float32)>

In [22]:
# here
# y[0][0] is the value of the 1st formula, i.e. x_0/pi for the first data batch
# y[0][6] is the value of the 6th formula, i.e. cos(x_0)*cos(x_1)/pi for the first data batch
# y[1][0] is the value of the 1st formula for the second data batch
y = resolved_formulas(x)
y

<tf.Tensor: shape=(10, 9), dtype=float32, numpy=
array([[-0.21716157, -0.10619579,  0.04276618,  0.04736071,  0.48331952,
         0.23343895,  0.3119561 ,  0.29804844,  0.0164886 ],
       [ 0.59020096,  0.4200015 , -0.28928185,  0.2161739 ,  0.002944  ,
        -0.02213264,  0.15224782,  0.04865808,  0.24767244],
       [-0.36294067,  0.48311746, -0.02880587,  0.54221314, -0.09498605,
         0.00704349, -0.04191731,  0.01680554, -0.04022943],
       [-0.26110584, -0.26687625,  0.47632015,  0.04778745, -0.22746024,
         0.14515574,  0.02339189,  0.01581871,  0.23773521],
       [ 0.23285633, -0.04184684,  0.3378908 ,  0.8636676 ,  0.04883707,
         0.23482525, -0.14117381,  0.15385267, -0.2861565 ],
       [ 0.03312568, -0.05346922, -0.12096003,  0.19232003,  0.3754918 ,
         0.31213173,  0.2432694 ,  0.29144162,  0.09987251],
       [-0.3832964 ,  0.38345712,  0.01602725,  0.23215058, -0.06237936,
         0.04085069,  0.23703933,  0.11381204,  0.23279722],
       [ 0.41

# The PQC class
- It accepts the `model_circuit` and `data_circuit` as input
- `model_circuit` contains the weights of your pqc layer so this circuit should not contain composite formulas in its gate operations
- `data_circuit` is the circuit that transforms the classical input data into the input quantum state of the `model_circuit`, it can contain composite formmulas in its gate operations which correspond to the so-called "feature maps"

What it does:
- It flattens the `data_circuit` so that all formulas inside the circuit turns into symbols. This is important because tfq simulator (qsim) does not support resolving sympy formulas.
- It compiles the formulas inside the `data_circuit` in terms of a lambda function of composite tensorflow operations when the class is initialzed. 
- When the class is called, it transforms the `inputs` tensor into a tensor containing the resolved values of the formulas of the `data_circuit`. The tensor is then clipped to the weight tensor and passed as `symbol_values` to the tfq simulator.

In [23]:
from quple.interface.tfq.layers.pqc import PQC

In [26]:
qubits = quple.get_circuit_qubits(data_circuit)
readout = [cirq.Z(qubit) for qubit in qubits]

In [27]:
pqc_layer = PQC(model_circuit, data_circuit, readout)

In [28]:
x = tf.random.normal((batchsize, feature_dimension))
pqc_layer(x)

<tf.Tensor: shape=(10, 5), dtype=float32, numpy=
array([[-0.5925325 ,  0.04048223, -0.3626655 , -0.19755554,  0.8203235 ],
       [-0.5925317 ,  0.04048201, -0.36266494, -0.19755527,  0.82032245],
       [-0.5925322 ,  0.04048229, -0.36266533, -0.19755542,  0.82032305],
       [-0.5925321 ,  0.04048215, -0.3626653 , -0.19755547,  0.820323  ],
       [-0.59253234,  0.0404822 , -0.36266539, -0.19755545,  0.8203232 ],
       [-0.5925323 ,  0.04048217, -0.36266539, -0.19755557,  0.82032317],
       [-0.59253234,  0.04048224, -0.3626653 , -0.19755542,  0.82032317],
       [-0.5925324 ,  0.0404823 , -0.36266527, -0.19755548,  0.82032317],
       [-0.5925325 ,  0.04048236, -0.36266553, -0.19755554,  0.8203236 ],
       [-0.59253246,  0.04048225, -0.36266539, -0.19755557,  0.8203233 ]],
      dtype=float32)>

In [30]:
model = tf.keras.Sequential()
input_layer = tf.keras.layers.Input(shape=(feature_dimension), dtype=tf.float32)
model.add(input_layer)
model.add(pqc_layer)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
pqc (PQC)                    (None, 5)                 28        
Total params: 28
Trainable params: 28
Non-trainable params: 0
_________________________________________________________________


In [31]:
model(x)

<tf.Tensor: shape=(10, 5), dtype=float32, numpy=
array([[-0.5925325 ,  0.04048223, -0.3626655 , -0.19755554,  0.8203235 ],
       [-0.5925317 ,  0.04048201, -0.36266494, -0.19755527,  0.82032245],
       [-0.5925322 ,  0.04048229, -0.36266533, -0.19755542,  0.82032305],
       [-0.5925321 ,  0.04048215, -0.3626653 , -0.19755547,  0.820323  ],
       [-0.59253234,  0.0404822 , -0.36266539, -0.19755545,  0.8203232 ],
       [-0.5925323 ,  0.04048217, -0.36266539, -0.19755557,  0.82032317],
       [-0.59253234,  0.04048224, -0.3626653 , -0.19755542,  0.82032317],
       [-0.5925324 ,  0.0404823 , -0.36266527, -0.19755548,  0.82032317],
       [-0.5925325 ,  0.04048236, -0.36266553, -0.19755554,  0.8203236 ],
       [-0.59253246,  0.04048225, -0.36266539, -0.19755557,  0.8203233 ]],
      dtype=float32)>